# Build Model for Training, and Test accuracy

In [331]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
import cPickle as pickle
import tensorflow as tf

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline
np.set_printoptions(threshold=np.inf)

### Deserialize

In [332]:
train_pickle_file = 'SVHN_basic_train.pickle'

with open(train_pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_labels = save['train_image_labels']
  train_dataset = save['train_dataset']
  del save  # hint to help gc free up memory
  print('Training set', train_labels.shape, train_dataset.shape)
print(train_labels[0:2,:])

Training set (33402, 6) (33402, 64, 64)
[[ 2. -1. -1. -1.  1.  9.]
 [ 2. -1. -1. -1.  2.  3.]]


In [333]:
test_pickle_file = 'SVHN_basic_test.pickle'

with open(test_pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_labels = save['test_image_labels']
  test_dataset = save['test_dataset']
  del save  # hint to help gc free up memory
  print('Test set', test_labels.shape, test_dataset.shape)

Test set (13068, 6) (13068, 64, 64)


####TODO: Randomize and Shuffle
####TODO: Use Boosting
####TODO: Use Simulated Annealing / Decay

###Reformat

In [334]:
### Reformat dataset to 1 channel
image_size=64
num_channels=1 #greyscale

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels))\
        .astype(np.float32)
  return dataset

train_dataset = reformat(train_dataset)
test_dataset = reformat(test_dataset)
print("Reformatted shapes of datasets\n")
print("train_dataset.shape:",train_dataset.shape,", train_labels.shape:",train_labels.shape)
print("test_dataset.shape:", test_dataset.shape,", test_labels.shape:",test_labels.shape)

Reformatted shapes of datasets

train_dataset.shape: (33402, 64, 64, 1) , train_labels.shape: (33402, 6)
test_dataset.shape: (13068, 64, 64, 1) , test_labels.shape: (13068, 6)


### Define Accuracy

In [335]:
def accuracy(predictions, labels):
    label_count=labels.shape[0]
    predictions=predictions.astype(int)
    print("predictions:\n",predictions)
    print("labels:\n", labels)
    len_preds=np.argmax(predictions[0,:,:],axis=1)
    digit_1_preds=np.argmax(predictions[1,:,:],axis=1)
    digit_2_preds=np.argmax(predictions[2,:,:],axis=1)
    digit_3_preds=np.argmax(predictions[3,:,:],axis=1)
    digit_4_preds=np.argmax(predictions[4,:,:],axis=1)
    digit_5_preds=np.argmax(predictions[5,:,:],axis=1)
    
    #print("len preds:", len_preds)
    #print(" digit_1_preds",digit_1_preds)
    #print(" digit_2_preds",digit_2_preds)
    #print(" digit_3_preds",digit_3_preds)
    #print(" digit_4_preds",digit_4_preds)
    #print(" digit_5_preds",digit_5_preds)
    
    len_accuracy=len_preds==labels[:,0]
    #print("len_accuracy:",len_accuracy.shape)
    digit_1_accuracy=digit_1_preds==labels[:,1]
    print("digit_1_accuracy:",digit_1_accuracy.shape, digit_1_accuracy)
    digit_2_accuracy=digit_2_preds==labels[:,2]
    digit_3_accuracy=digit_3_preds==labels[:,3]
    digit_4_accuracy=digit_4_preds==labels[:,4]
    digit_5_accuracy=digit_1_preds==labels[:,5]
    
    
    complete_accuracy=np.concatenate((len_accuracy.reshape(1,label_count),digit_1_accuracy.reshape(1,label_count),\
                                      digit_2_accuracy.reshape(1,label_count),digit_3_accuracy.reshape(1,label_count), \
                                      digit_4_accuracy.reshape(1,label_count),digit_5_accuracy.reshape(1,label_count)), axis=0).T
    print("complete_accuracy:\n",complete_accuracy.shape, complete_accuracy)
    return 100.0 * (np.sum([np.all(row) for row in complete_accuracy])) / len(labels)
    #return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
    #      / predictions.shape[0])

#### Create Model

In [336]:
batch_size = 128
patch_size = 5
depth = 8
num_hidden = 8
num_labels = 10 # 10 for 0-9
len_labels=6 # no digit, 5 for lenghts0-5, and >5


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, len_labels))
  tf_digit_masks_1=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_2=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_3=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_4=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_5=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  cnv_lyr1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.01))
  cnv_lyr1_biases = tf.Variable(tf.constant(0.01, shape=[depth]))
  cnv_lyr2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  cnv_lyr2_biases = tf.Variable(tf.constant(0.01, shape=[depth]))
  cnv_lyr3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, num_hidden], stddev=0.01))
  cnv_lyr3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
  

  length_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, len_labels], stddev=0.1))
  length_biases = tf.Variable(tf.constant(0.01, shape=[len_labels]))

  digit1_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, num_labels], stddev=0.1))
  digit1_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

  digit2_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, num_labels], stddev=0.1))
  digit2_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

  digit3_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, num_labels], stddev=0.1))
  digit3_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

  digit4_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, num_labels], stddev=0.1))
  digit4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

  digit5_weights = tf.Variable(tf.truncated_normal([num_hidden * depth * depth * 16, num_labels], stddev=0.1))
  digit5_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

  def max_pool_2x2(x):
     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
  
  # Model.
  def model(data, digit_masks_1,digit_masks_2,digit_masks_3,digit_masks_4,digit_masks_5, is_train):
    print("data:",data.get_shape())
    conv = tf.nn.conv2d(data, cnv_lyr1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr1_biases)
    #h_pool1 = max_pool_2x2(hidden)
    print("hidden:",hidden.get_shape())
    
    conv = tf.nn.conv2d(hidden, cnv_lyr2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr2_biases)
    h_pool2 = max_pool_2x2(hidden)
    print("h_pool2:",h_pool2.get_shape())

    conv = tf.nn.conv2d(h_pool2, cnv_lyr3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr3_biases)
    h_pool3 = max_pool_2x2(hidden)
    print("h_pool3:",h_pool3.get_shape())

    shape = h_pool3.get_shape().as_list()
    #print("shape:",shape)
    reshape = tf.reshape(h_pool3, [shape[0], shape[1] * shape[2] * shape[3]])
    
    #digit_masks=(1-np.isnan(labels).astype(int))
    #digit_masks=[(1-x) for x in labels if x != 'nan']
    print("reshape:", reshape)
    #print("tf_digit_masks:", digit_masks_1,digit_masks_2,digit_masks_3,digit_masks_4,digit_masks_5)
  
    print()
    logit_length = tf.matmul(reshape, length_weights) + length_biases
    print("digit1_weights.shape: ",digit1_weights.get_shape())
    print("digit1_biases.shape: ",digit1_biases.get_shape())

    logit_1 = (tf.matmul(reshape, digit1_weights) + digit1_biases)*(digit_masks_1 if is_train==True else 1)
    logit_2 = (tf.matmul(reshape, digit2_weights) + digit2_biases)*(digit_masks_2 if is_train==True else 1)
    logit_3 = (tf.matmul(reshape, digit3_weights) + digit3_biases)*(digit_masks_3 if is_train==True else 1)
    logit_4 = (tf.matmul(reshape, digit4_weights) + digit4_biases)*(digit_masks_4 if is_train==True else 1)
    logit_5 = (tf.matmul(reshape, digit5_weights) + digit5_biases)*(digit_masks_5 if is_train==True else 1)
    

    return logit_length, logit_1,logit_2,logit_3,logit_4,logit_5
  
  # Training computation.
  logit_len, logit_digit_1,logit_digit_2,logit_digit_3,logit_digit_4,logit_digit_5 \
    = model(tf_train_dataset, tf_digit_masks_1, tf_digit_masks_2, tf_digit_masks_3, tf_digit_masks_4, \
            tf_digit_masks_5, True)
    
  loss_len     = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_len, tf_train_labels[:,0])
  loss_digit_1 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_1, tf_train_labels[:,1])
  loss_digit_2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_2, tf_train_labels[:,2])
  loss_digit_3 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_3, tf_train_labels[:,3])
  loss_digit_4 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_4, tf_train_labels[:,4])
  loss_digit_5 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_5, tf_train_labels[:,5])
    
  loss = tf.reduce_mean(loss_len + loss_digit_1 + loss_digit_2 + loss_digit_3 + loss_digit_4 + loss_digit_5)
    
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.99, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  print("logit_len:",logit_len)
  print("logit_digit_1:",logit_digit_1)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.pack([ tf.nn.softmax(logit_len),
                               tf.nn.softmax(logit_digit_1),
                               tf.nn.softmax(logit_digit_2),
                               tf.nn.softmax(logit_digit_3),
                               tf.nn.softmax(logit_digit_4),
                               tf.nn.softmax(logit_digit_5)
                             ])
 

data: (128, 64, 64, 1)
hidden: (128, 64, 64, 2)
h_pool2: (128, 16, 16, 2)
h_pool3: (128, 8, 8, 8)
reshape: Tensor("Reshape:0", shape=(128, 512), dtype=float32)

digit1_weights.shape:  (512, 11)
digit1_biases.shape:  (11,)
logit_len: Tensor("add_3:0", shape=(128, 11), dtype=float32)
logit_digit_1: Tensor("mul:0", shape=(128, 11), dtype=float32)


In [337]:
def explode(batch_labels_arr):
    target_row_count=batch_labels_arr.shape[0]
    target_col_count=11
    total_arr_count=5
    digit_mask_array = np.ndarray(shape=(total_arr_count, target_row_count, target_col_count),dtype=np.int32)
    digit_mask_array[:,:,:]=int(0)
    #print("batch_labels_arr: \n",batch_labels_arr.shape)
    #print(digit_mask_array.shape)
    for row in range(target_row_count):
        for digit_col in range(total_arr_count):
            digit_val=batch_labels_arr[row,digit_col+1]
            #print("batch_labels_arr[",row,",",digit_col,"]=",digit_val)
            if digit_val == -1 or digit_val > 9: 
                digit_mask_array[digit_col, row, 0] = 1
            else:
                digit_mask_array[digit_col, row, digit_val.astype(int)+1] = 1
    #print("digit_mask_array:\n",digit_mask_array)
    return digit_mask_array[0,:,:],digit_mask_array[1,:,:],digit_mask_array[2,:,:],digit_mask_array[3,:,:],digit_mask_array[4,:,:],
    
#print(train_labels[0:5,:])        
#explode(train_labels[0:5,:])

In [338]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print("offset:", offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :,:]
    #print("batch_data:",batch_data.shape)
    batch_labels = train_labels[offset:(offset + batch_size), :].astype(int)
    digit_1_mask, digit_2_mask,digit_3_mask,digit_4_mask,digit_5_mask = explode(batch_labels)
    #batch_masks=np.array([item == item for item in batch_labels]).astype(int)
    #print("digit_5_mask:",digit_5_mask)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_digit_masks_1 : digit_1_mask ,\
                tf_digit_masks_2 : digit_2_mask, tf_digit_masks_3 : digit_3_mask, tf_digit_masks_4 : digit_4_mask, \
                tf_digit_masks_5 : digit_5_mask}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step > 1 and step % 10000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
     # print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 10000: 5.966691
predictions:
 [[[0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0]

KeyboardInterrupt: 